In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:

data_path = r"data/interim/loan_data_clean.csv" 
preprocessed_path = "data/processed/loan_data_preprocessed.csv"
df = pd.read_csv(data_path)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Missing values:\n", df.isnull().sum()[df.isnull().sum() > 0])
print("Duplicate rows:", df.duplicated().sum())


Shape: (38480, 37)
Columns: ['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'repay_fail']
Missing values:
 loan_amnt                     1
funded_amnt                   1
funded_amnt_inv               1
installment                   1
emp_length                  993
annual_inc                    2
delinq_2yrs                   1
inq_last_6mths                1
mths_since_last_delinq    24363
open_acc                      1
pub_rec                       1
revol_bal                     4
revol_util       

In [8]:
#Handling Missing Values

numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

categorical_cols = df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

df.to_csv(preprocessed_path, index=False)

print("Missing values handled")
print(df.isnull().sum().sum())


Missing values handled
0


In [ ]:
# Data Type Conversion

if "int_rate" in df.columns:
    df["int_rate"] = df["int_rate"].str.rstrip("%").astype(float)
if "revol_util" in df.columns:
    df["revol_util"] = df["revol_util"].str.rstrip("%").astype(float)

# Convert date columns
date_cols = ["issue_d", "last_pymnt_d", "next_pymnt_d", "last_credit_pull_d"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

print(" Data types converted")
print(df.dtypes)

# Save
df.to_csv(preprocessed_path, index=False)
print(" Saved after type conversion")

In [ ]:
#Encode Features


# Ordinal encoding for emp_length
emp_mapping = {
    "< 1 year": 0, "1 year": 1, "2 years": 2, "3 years": 3, "4 years": 4, "5 years": 5,
    "6 years": 6, "7 years": 7, "8 years": 8, "9 years": 9, "10+ years": 10
}
if "emp_length" in df.columns:
    df["emp_length"] = df["emp_length"].map(emp_mapping)

# One-hot encoding for selected categorical variables
onehot_cols = ["home_ownership", "purpose", "verification_status", "addr_state"]
existing_cols = [col for col in onehot_cols if col in df.columns]
df = pd.get_dummies(df, columns=existing_cols, drop_first=True)

print(" Categorical features encoded")
df.head()

df.to_csv(preprocessed_path, index=False)
print(" Saved after encoding")

In [ ]:
#Feature Engineering


# Example features
if all(col in df.columns for col in ["annual_inc", "loan_amnt"]):
    df["income_to_loan_ratio"] = df["annual_inc"] / df["loan_amnt"]

if all(col in df.columns for col in ["revol_bal", "annual_inc"]):
    df["credit_utilization_ratio"] = df["revol_bal"] / df["annual_inc"]

if "issue_d" in df.columns:
    today = pd.to_datetime("today")
    df["loan_age_in_months"] = (today - df["issue_d"]).dt.days // 30

if all(col in df.columns for col in ["total_pymnt", "loan_amnt"]):
    df["payment_to_loan_ratio"] = df["total_pymnt"] / df["loan_amnt"]

print("Feature engineering done")
df.head()

# Save
df.to_csv(preprocessed_path, index=False)
print("Saved after feature engineering")